<img vspace="33px" align="right" src="https://www.munich-startup.de/wp-content/uploads/2019/03/TUM_logo-440x236.png" width="120px"/>
<h1>Sign Language Production</h1>
<h3>Applied Deep Learning for NLP</h3>
<p><b>Diego Miguel Lozano</b> | <b>Wenceslao Villegas Marset</b></p>
<p>March 9<sup>th</sup>, 2022</p>

---

# Table of contents

> ### [1. Introduction](#section_1)
>> [**1.1 What is Sign Language Production (SLP)?**](#section_1_1)<br>
>> [**1.2 What is the starting point for our project?**](#section_1_2)

# 0. Set-up and Imports

In [7]:
import sys
import logging
import torch
from pathlib import Path
from pprint import pprint
from torch import nn
from transformers import AutoTokenizer, AutoModelForMaskedLM, logging
from transformers import logging

<a id='section_1'></a>
# 1. Introduction

<a id='section_1_1'></a>
### 1.1 What is Sign Language Production (SLP)?

Sign Language Production focuses on translating spoken languages into sign languages and viceversa. According to the World Health Organization (WHO), in 2020 there were more than 466 million deaf people in the world [[1]](#ref_1). This area could be of great help for the hearing-impared community, being for that necessary the development of techniques for both recognition and production of sign languages.

While the Sign Language Recognition has seen numerous advancements in the last years [[2](#ref_2), [3](#ref_3), [4](#ref_4), [5](#ref_5), [6](#ref_6), [7](#ref_7)], Sign Language Production is still a very challenging task, since it involves an interpretation between visual and linguistic information [[8]](#ref_8).

<a id='section_1_2'></a>
### 1.2 What is the starting point for our project?

As we just mentioned, SLP is complex and far from being solved. Nevertheless, there have recently been promising developments, such as the application of Transformer architectures to SLP, what has come to be called "Progressive Transformers."

In this project, we take the [source code](https://github.com/BenSaunders27/ProgressiveTransformersSLP) for the paper "Progressive Transformers for End-to-End Sign Language Production" [[9]](#ref_9) as the starting point.

**TODO**: define exactly what the scope of our project is.

---

# A brief intro to the "Progressive Transformers for SLP" project

In this section, we will quickly explain the main aspects of Progressive Transformers project. If we had to summarize it in only three points, these would be the following: counter decoding, two different approaches –Text-to-Gloss-to-Pose (T2G2P) and Text-to-Pose (T2P)–, and data augmentation.

## Counter decoding

One of the main challenges of SLP is that the output has to maintain certain continuity. The predicted pose in a video frame has to flow naturally from the previous one, and analogously for the frames that follow. This is achieved in the following manner: the model not only predicts the sign pose, but also a "counter". This counter is nothing else but real number in the interval [0, 1]. This value increases monotonically from 0 to 1.0, marking thus the begining and end of sequence, respectively.

<img width="600px" src="./images/counter-decoding.jpg" alt="Counter Decoding"/>
<br>
<div align="center"><i>Representation of counter decoding.</i></div>
<br>

---

**ℹ️ Question: Why not simply use an BOS token and EOS?**

**💡 Answer:** Begining of Sentence (BOS) and End of Sentence (EOS) tokens work well with sentences, but when producing video as we mentioned before we need something more than just marking the begging and end of it. Therefore, the counter serves both as an BOS and EOS and captures information about the flow of the video.

---


### Two different approaches

In the paper, they experimented with two different approaches: T2G2P and T2P:

<img width="600px" src="./images/T2G2P-vs-T2P.png" alt="T2G2P vs T2P Architectures"/>
<br>
<div align="center"><i>Architecture details of (a) Symbolic and (b) Progressive Transformers. (ST: Symbolic Transformer, PT: Progressive Transformer, PE: Positional Encoding, CE: Counter Embedding, MHA: Multi-Head Attention) <a href="#ref_10">[10]</a>.</i></div>

In both cases, the models follow the architecture introduced in "Transfomers is All You Need" <a href="#ref_11">[10]</a>.

In the first approach –T2G2P– glosses are produced from the input tokens in a first step. Then, this glosses serve as input for another transformer, which then translates the glosses into sign poses.

The second –T2P– is an end-to-end approach, in which the text is directly translated into sign poses.

### Data augmentation

Finally, the paper explores some data augmentation techniques to determine whether they improve the base model. These augmentations where only carried out with the T2G2P architecture.

- **Future Prediction**: this type of augmentation forces the model to predict the next 10 frames from the current time step, instead of just the next frame. In this way, the model cannot just copy the previous time step, which effectively improves performance over the base architecture.


- **Just Counter**: in this case only the counter values are provided as target input to the model, omitting the 3D skeleton joint coordinates. Again, this has shown to improve results.


- **Gaussian Noise**: the last augmentation method consists in adding Gaussian noise to the skeleton pose sequences during training. This makes the model more robust to prediction inputs.


The following table collects the results of the previous augmentation approaches:

<img width="700px" src="./images/augmentation-results.png" alt="Data Augmentation Results"/>
<br>
<div align="center"><i>The best BLEU-4 performance comes from a combination of future prediction and Gaussian noise augmentation. The model must learn to cope with both multi-frame prediction and a noisy input, building a firm robustness to drift <a href="#ref_10">[10]</a>.</i></div>

### #TODO: document main aspects of their code (Wences?)

---

#  Using pre-trained embeddings

The original project trains embeddings from scratch. As we have learned during the seminar, the use of pretrained embeddings can effectively improve the performance of models, especially in situations where data is scarce (which is our case).

Let's first see how the embedding initialization is happening in the original source code.

The first thing that happens when beginning the training is the data loading. The vocabulary of the model is initalized differently depending if we are using T2G2P or T2P.

- **Bulding vocabulary in T2G2P**: in this case, the vocabulary is taken from a file [`src_vocab`](https://github.com/BenSaunders27/ProgressiveTransformersSLP/blob/master/Configs/src_vocab.txt) that we will analyze a bit more in depth later.


- **Bulding vocabulary in T2P**: when using the End-to-End approach (Text-to-Pose), the vocabulary is built from the training input data.

<br>

In both cases, the function [`build_vocab()`](https://github.com/BenSaunders27/ProgressiveTransformersSLP/blob/adbd3e9ea9f1b20063d84021a0d6eb9a124ebb87/vocabulary.py#L130-L187) is used. Let's take a look at it.


```python
if vocab_file is not None:
    # load it from file
    vocab = Vocabulary(file=vocab_file)
else:
    ...
```

<br>

First of all, if we have a vocabulary file (like in the case of T2G2P), we initialize the vocabulary from it, as we already mentioned.


```python
def _from_file(self, file: str) -> None:
        """
        Make vocabulary from contents of file.
        File format: token with index i is in line i.

        :param file: path to file where the vocabulary is loaded from
        """
        tokens = []
        with open(file, "r") as open_file:
            for line in open_file:
                tokens.append(line.strip("\n"))
        self._from_list(tokens)
```

This function simply reads the vocabulary file line by line. Since each line contains only one token, there is no further processing to be done.

If there is no input vocabulary file, the tokens are extracted from the training dataset.

Let's run some code to better visualize this.

## Inside the original SLP model vocab

As we have already mentioned, the original project that we use as starting point provides a plain-text file [`src_vocab`](https://github.com/BenSaunders27/ProgressiveTransformersSLP/blob/master/Configs/src_vocab.txt) containing the vocabulary for which embeddings will then be trained.

Before jumping in and trying to directly use our pretrained embeddings, it is sensible to first analyze a bit how things work in the original project.

In [8]:
code_dir = Path("./ProgressiveTransformersSLP")

if str(code_dir.resolve()) not in sys.path:
    sys.path.insert(0, str(code_dir.resolve()))  # just so that imports can be resolved

from ProgressiveTransformersSLP.vocabulary import Vocabulary, build_vocab

In [9]:
# Path to the vocabulary file
vocab_file = code_dir/Path("Configs/src_vocab.txt")

# Build vocabulary
vocabulary = Vocabulary(file=vocab_file)

# Get all the tokens in the built vocabulary
tokens = [token for token in vocabulary.itos]

# We will select some tokens that are worth analyzing
selected_tokens = (tokens[1:5] + [tokens[172]] + tokens[531:541] +
                   tokens[868:870] + tokens[718:725] + tokens[1085:1088])
pprint(selected_tokens)

['<unk>',
 '<pad>',
 '<s>',
 '</s>',
 'HEISS',
 'AUSWAEHLEN',
 'BALD',
 'BEKOMMEN',
 'BITTE',
 'BODENSEE',
 'BRITANNIEN',
 'CHAOS',
 'DAMEN',
 'DAUERND',
 'DUENN',
 'J+L+I',
 'K+R+E+T+A',
 'neg-EINFLUSS',
 'neg-FUEHLEN',
 'neg-GEWITTER',
 'neg-HOEHE',
 'neg-IMMER',
 'neg-KOMMEN',
 'neg-MEHR',
 'negalp-BRAUCHEN',
 'negalp-GIBT',
 'negalp-MUSS']


From the previous vocabulary, there are three aspects that are worth mentioning:

- Words such as `AUSWAEHLEN`, `DUENN` and `HEISS` give us a hint that **normalization** is used. A popular algorithm for German normalization is the [German2 snowball algorithm](https://snowballstem.org/algorithms/german2/stemmer.html) which defines the following mappings:
  - 'ß' is replaced by 'ss'.
  - 'ä', 'ö', 'ü' are replaced by 'a', 'o', 'u', respectively.
  - 'ae' and 'oe' are replaced by 'a', and 'o', respectively.
  - 'ue' is replaced by 'u', when not following a vowel or q.


- As we saw during the seminar lectures, the **special tokens** `<unk>`, `<pad>`, `<s>`, `</s>` are also included in the dictionary. These tokens mark unknown words, padding, beginning of sequence (BOS), and end of sequence (EOS), respectively.


- Some of the words in the vocabulary include the prefixes `neg-` and `negalp-`. We could guess that `neg-` simply means that the word is negated, e.g., `neg-GENUG`≡ `NICHT GENUG`, but what about the `negalp-` prefix? And also, what do words such as `J+L+I` and `K+R+E+T+A` mean? A look to the paper of the RWTH-PHOENIX-Weather dataset [[10]](#ref_10) (the first version of the dataset used to train the model) gives us the answer:


<img width="400px" src="./images/RWTH-PHOENIX-Weather-Annotation-Scheme.png" alt="RWTH-PHOENIX-Weather Annotation Scheme"/>
<br>
<div align="center"><i>Source <a href="#ref_10">[10]</a>.</i></div>

So in reality `neg-` means "signs negated by headshake" and `negalp-` "signs negated by the alpha[betical] rule" <sup>[1](note_1)</sup>. Words such as `K+R+E+T+A` are words (finger) spelled letter by letter.

Interestingly enough, none of the other types of tokens appear in the source dictionary.

---

<a id='note_1'><sup>1</sup></a> In Sign Language, there are several ways of negating words. One of these ways is using a side-to-side headshake or a frown expression. Also, some verbs have their own negated forms, which is what `negalp-` indicates here [[11]](#ref_11).

---

**ℹ️ Question: But... where is this vocabulary coming from?**

**💡 Answer:** As it turns out, this vocabulary is simply made up of **glosses**. As we mentioned before, the original project proposes two ways of carrying out the translations from text to sign language. That also explains why when going for the T2P approach we don't use this file.

---

## BERT it up!

If something is clear is that BERT's dictionary will not contain glosses, let alone glosses specifically tailored to SLP.

But then, how does BERT's vocabulary looks like? Let's take a look!

Fortunately, Hugging Face's great API has got us covered: tokenizers expose their vocabulary through the method `get_vocab()`. Let's try with the model [`bert-base-german-cased`](https://huggingface.co/bert-base-german-cased).

In [10]:
# Dismiss expected "Some weights of the model checkpoint at..." warning
# when loading a pretrained model.
logging.set_verbosity_error()

# Load tokenizer and model from pretrained model
tokenizer = AutoTokenizer.from_pretrained("bert-base-german-cased")
model = AutoModelForMaskedLM.from_pretrained("bert-base-german-cased")

In [11]:
# Print some words of its dictionary
pprint(list(tokenizer.get_vocab().items())[:40])

[('Job', 6323),
 ('Columb', 11908),
 ('##yz', 13087),
 ('##rucken', 17229),
 ('Eltern', 2579),
 ('Zuge', 3960),
 ('##sichtlichen', 18282),
 ('hebt', 20335),
 ('Isar', 20735),
 ('Cant', 21447),
 ('stets', 4861),
 ('ca', 3678),
 ('Turn', 8862),
 ('##umpf', 14368),
 ('[unused2]', 27001),
 ('Cru', 12751),
 ('einzel', 2234),
 ('abzuweisen', 9137),
 ('betriebliche', 18002),
 ('##vest', 2412),
 ('##eifen', 15938),
 ('frühzeitig', 21442),
 ('Belehrung', 26193),
 ('Restaurierung', 22282),
 ('##zeitungen', 23664),
 ('Institute', 9938),
 ('##anni', 13408),
 ('rechteck', 16935),
 ('##bald', 12308),
 ('Verantwort', 4735),
 ('Beiträgen', 21922),
 ('##Bad', 22606),
 ('##zeitpunkt', 21135),
 ('gleichnamigen', 8254),
 ('verfügte', 11048),
 ('##chen', 280),
 ('E', 55),
 ('##igsten', 12466),
 ('##östlich', 3286),
 ('##ption', 9346)]


Above, we can see each token with its corresponding ID (just as we saw in the seminar lectures). However, there are two things that catch our attention...

---

**ℹ️ Question: Why do some tokens start with "##"?**

**💡 Answer:** Well, this is just a way of indicating that this token is "non-initial", i.e., originally it belonged to a longer word. This way of tokenizing words comes from the [WordPiece](https://paperswithcode.com/method/wordpiece) algorithm, which is used by the BERT tokenizer.

---

**ℹ️ Question: What about the `[unused...]` tokens?**

**💡 Answer:** These are, unsurprisingly, tokens that are not used. However, they can come handy to add more words to the vocabulary:

> Just replace the "[unusedX]" tokens with your vocabulary. Since these were not used they are effectively randomly initialized. ([source](https://github.com/google-research/bert/issues/9#issuecomment-434796704))

---

As a side note, we would like to mention that someone took the time to explore BERT's vocabulary and wrote a great article about it. The article can be found at https://juditacs.github.io/2019/02/19/bert-tokenization-stats.html.

---

We might also want to take a look at the emeddings of a word:

In [24]:
token_id = tokenizer.get_vocab()["Frieden"]  # get token ID
embeddings = model.get_input_embeddings()(torch.tensor(token_id))
# Print only the 32 first values
print(embeddings[:32])
print("\nDimensions:", len(embeddings))

tensor([ 0.0445,  0.0229, -0.0298, -0.0048,  0.0607, -0.0152, -0.0425, -0.0234,
        -0.0486, -0.0170,  0.0113,  0.0774, -0.0335,  0.0334, -0.0383,  0.0304,
         0.0650,  0.0320, -0.0085, -0.0445,  0.0263, -0.0207, -0.0087, -0.0217,
        -0.0547,  0.0187, -0.0181,  0.0097,  0.0159,  0.0296, -0.0319,  0.0168],
       grad_fn=<SliceBackward0>)

Dimensions: 768


### Alright, so how do we plug in the pretrained embeddings?

In order to use

In [ ]:
tokenizer("Unglaublich")['input_ids']

In [20]:
tokenizer.get_vocab()["[UNK]"]

2

# References

<a id='ref_1'>[1]</a> WHO: World Health Organization. Deafness and hearing loss. http://www.who.int/mediacentre/factsheets/fs300/en/, 2021

<a id='ref_2'>[2]</a> Razieh Rastgoo, Kourosh Kiani, and Sergio Escalera. Multimodal deep hand sign language recognition in still images using restricted boltzmann machine. Entropy, 20, 2018.

<a id='ref_3'>[3]</a> Razieh Rastgoo, Kourosh Kiani, and Sergio Escalera. Hand sign language recognition using multi-view hand skeleton. Expert Systems With Applications, 150, 2020.

<a id='ref_4'>[4]</a> Razieh Rastgoo, Kourosh Kiani, and Sergio Escalera. Video based isolated hand sign language recognition using a deep cascaded model. Multimedia Tools And Applications, 79:22965–22987, 2020.

<a id='ref_5'>[5]</a> Razieh Rastgoo, Kourosh Kiani, and Sergio Escalera. Hand pose aware multimodal isolated sign language recognition. Multimedia Tools And Applications, 80:127–163, 2021

<a id='ref_6'>[6]</a> Mark Borg and Kenneth P. Camilleri. Phonologically-meaningful sub-units for deep learning-based sign language recognition. ECCV, 2020

<a id='ref_7'>[7]</a> Agelos Kratimenos, Georgios Pavlakos, and Petros Maragos. 3d hands, face and body extraction for sign language recognition. ECCV, 2020.

<a id='ref_8'>[8]</a> Razieh Rastgoo and Kourosh Kiani and Sergio Escalera and Mohammad Sabokrou. Sign Language Production: A Review. 2021.

<a id='ref_9'>[9]</a> Saunders, Ben and Camgoz, Necati Cihan and Bowden, Richard. Progressive Transformers for End-to-End Sign Language Production. https://www.ecva.net/papers/eccv_2020/papers_ECCV/papers/123560664.pdf. ECCV, 2020.

<a id='ref_10'>[10]</a> J. Forster, C. Schmidt, T. Hoyoux, O. Koller, U. Zelle, J. Piater, and H. Ney. RWTH-PHOENIX-Weather: A Large Vocabulary Sign Language Recognition and Translation Corpus. https://www-i6.informatik.rwth-aachen.de/publications/download/773/Forster-LREC-2012.pdf In Language Resources and Evaluation (LREC), pages 3785-3789, Istanbul, Turkey, May 2012. 

<a id='ref_11'>[11]</a> Attention Is All You Need. Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin. https://arxiv.org/pdf/1706.03762.pdf, June 2017.

<a id='ref_11'>[12]</a> 

<a id='ref_11'>[13]</a> 

<a id='ref_11'>[14]</a> Handspeak. Negation in Sign Language. https://www.handspeak.com/learn/index.php?id=156, 2022.